# 📘 Notebook 01: Data Collection
# Emotional Geography of Books – Shraddha

In [6]:
import requests
import json
import os
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from pathlib import Path
from time import sleep
import kagglehub

In [9]:
df

""
